# Демонстрация инструмента последовательного снятия снимков агрегированных данных из потоковых данных

Импортируем необходимые библиотеки.

In [1]:
import time

import requests
import IPython.display

import constants
from _development.tests.test_utilities import print_response, print_tables_side_by_side
from _development.tests.subprocess_utilities import start_subprocess, terminate_shell_subprocess

Запустим генератор потоковых данных.

Генератор потоковых данных применяет изменения к таблице базы данных PostgreSQL. Оттуда данные выгружаются в Apache Kafka инструментом CDC Debezium. Далее данные поступают в разработанный инструмент с помощью коннектора DebeziumSource.

In [2]:
stream_generator = start_subprocess('python stream_generator.py')

Запустим разработанный инструмент.

In [3]:
main = start_subprocess('python ../../main.py')

Зададим имя топика Apache Kafka, откуда инструмент будет получать данные.

In [4]:
topic_name = 'postgres_source.public.demo_table'

Подготовим JSON запроса подключения к источнику данных CREATE SOURCE.

In [5]:
create_source = {'query_type': 'CREATE SOURCE',
                 'name': 'demo_source', 'type': 'DebeziumSource',
                 'parameters': {'kafka_topic_name': topic_name, 'group_id': None, 'auto_offset_reset': 'earliest',
                                'bootstrap_servers': ['kafka:9092'], 'consumer_timeout_ms': 1000}}

Подготовим JSON запроса создания материализованного представления CREATE MATERIALIZED VIEW.



In [6]:
create_view = {'query_type': 'CREATE MATERIALIZED VIEW',
               'name': 'demo_view', 'view_source_name': 'demo_source', 'groupby_columns': ['a'],
               'parameters': {'extrapolation': True, 'extrapolation_method': 'linear'},
               'aggregates': [{'function': 'Sum', 'column': 'c'}, {'function': 'Avg', 'column': 'c'},
                              {'function': 'CountDistinctCBF', 'column': 'b', 'parameters': {'expected_element_count': 100, 'false_positive_probability': 0.01}}]}

Подготовим JSON запроса получения данных SELECT.

In [7]:
select = {'query_type': 'SELECT', 'name': 'demo_view', 'orderby': [['a', 'DESC']], 'format': 'tabulate'}

In [8]:
select_extrapolated = {'query_type': 'SELECT FORECASTED', 'name': 'demo_view', 'orderby': [['a', 'DESC']], 'format': 'tabulate', 'extrapolation_offset': 0.5}

In [9]:
time.sleep(2)

In [10]:
print_response(requests.post(f'http://127.0.0.1:{constants.SERVER_PORT}', json=create_source))

200 OK


In [11]:
print_response(requests.post(f'http://127.0.0.1:{constants.SERVER_PORT}', json=create_view))

200 OK


In [12]:
while True:
    select_response = requests.post(f'http://127.0.0.1:{constants.SERVER_PORT}', json=select).content.decode()
    select_extrapolated_response = requests.post(f'http://127.0.0.1:{constants.SERVER_PORT}', json=select_extrapolated).content.decode()
    print_tables_side_by_side(select_response, select_extrapolated_response, 'SELECT', 'SELECT FORECASTED')
    time.sleep(5)
    IPython.display.clear_output()

SELECT                                                        SELECT FORECASTED                                      
╭─────┬──────────┬──────────┬───────────────────────────╮     ╭─────┬──────────┬──────────┬───────────────────────────╮
│   a │   sum(c) │   avg(c) │   count_distinct_c_b_f(b) │     │   a │   sum(c) │   avg(c) │   count_distinct_c_b_f(b) │
├─────┼──────────┼──────────┼───────────────────────────┤     ├─────┼──────────┼──────────┼───────────────────────────┤
│   5 │  125.993 │  2.03215 │                   22.6269 │     │   5 │  126.166 │  2.02943 │                   22.6269 │
│   4 │  141.199 │  2.2774  │                   22.4585 │     │   4 │  141.25  │  2.27203 │                   22.5981 │
│   3 │  142.841 │  2.30388 │                   21.7869 │     │   3 │  143.589 │  2.30987 │                   21.7869 │
│   2 │  159.683 │  2.57553 │                   21.2854 │     │   2 │  160.625 │  2.58396 │                   21.2854 │
│   1 │  150.619 │  2.42935 │             

KeyboardInterrupt: 

In [ ]:
terminate_shell_subprocess(main)

In [ ]:
terminate_shell_subprocess(stream_generator)

In [ ]:
import kafka_utilities

kafka_utilities.delete_topic(topic_name)